In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
from __future__ import print_function
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt, gridspec
import plotutils

In [ ]:
basedir = '../benchmarks_py/sfi_scaling006'
DIR, COL, desc  = ('agents', 'num_agents', '# agents')
DIR, COL, desc  = ('memes', 'num_memes', '# memes')
DIR, COL, desc  = ('lexes', 'num_lexes', '# lexes')
#DIR, COL, desc  = ('lexesmemes', 'num_memes', '# memes/lexes')
#DIR, COL, desc  = ('lexesmemesagents', 'num_agents', '# memes/lexes/agents')
DO_SAVE            = False # whether to save pdf output

max_step = 0
data, error_files = plotutils.load_data_dir(basedir+'/'+DIR)

mutators = set(cfg['mutator_class'] for cfg, _ in data)
colvals  = set(float(cfg[COL])      for cfg, _ in data)

print("Loaded %d files" % len(data))
if len(error_files):
    print("Errors processing these files:")
    for ffname in error_files:
        print("  %s" % ffname)


In [ ]:
mutator_scaling_slopes = {}        
for pCol in ['Comprehension', 'AcceptanceRate', 'MeanGrammarDrift', 'AgentGrammarDrift', 'GrammarVar']:
    print("Doing %s" % pCol)
    plt.figure(figsize=(15,4*len(mutators)))
    gs = gridspec.GridSpec(len(mutators), 3)
    for mndx, m in enumerate(sorted(mutators)):
        plot_dfs = [ (cfg, df) 
                     for v in sorted(colvals) for (cfg, df) in data
                     if cfg['mutator_class'] == m and float(cfg[COL]) == v and len(df) > 1]

        plt.subplot(gs[mndx,0])
        for cfg, df in plot_dfs:
            plt.plot(df.Step, df[pCol], label=cfg[COL])
        plt.xlabel('Step')
        plt.ylabel(pCol)
        plt.semilogx()
        plt.title(m + '-'+pCol, fontsize=13)

        plt.subplot(gs[mndx,1])
        xs, ys = [],[]
        for cfg, df in plot_dfs:
            steps, vals = df.Step, df[pCol]
            if pCol == 'AcceptanceRate':
                ixs = np.flatnonzero(vals <= 1.02*vals.min())
                if len(ixs) and ixs[0]>1:
                    xs.append(float(cfg[COL]))
                    ys.append(steps[ixs[0]])
                ylabel = 'Steps until ~min Acceptance'
                
            elif pCol == 'Comprehension':
                use_ix = None
                cutoff = 0.25
                ixs = np.flatnonzero(df[pCol] >= cutoff)
                if len(ixs) and ixs[0]>1:
                    xs.append(float(cfg[COL]))
                    ys.append(df.Step.iloc[ixs[0]])
                ylabel = 'Steps to %d%% Comprehension' % int(cutoff*100)
                
            else:
                y = float(vals.max())
                if y != 0:
                    xs.append(float(cfg[COL]))
                    ys.append(y)
                ylabel = 'max %s reached'%pCol
        if len(xs):
            slope, _ = plotutils.plot_scatter(xs, ys, loglog=True, regression_line=True)
            if pCol == 'Comprehension':
                 mutator_scaling_slopes[m] = slope

        plt.xlabel(desc)
        plt.ylabel(ylabel)

        plt.subplot(gs[mndx,2])
        plt.semilogx()
        #plt.title(m)
        for cfg, df in plot_dfs:
            v = float(cfg[COL])
            vals = df[pCol]
            rescaled = df.Step/(v**mutator_scaling_slopes[m])
            plt.plot(rescaled, vals, label=cfg[COL])
            if v==21:
                plt.xlim([0, 2*np.max(rescaled)])
        plt.xticks([])
        plt.xlabel('Steps Rescaled ($Steps \\times %s^{-%0.2f}$)' % (desc.replace('#', '\\#'), mutator_scaling_slopes[m]))
        plt.ylabel('%s'%pCol)
        plt.legend(title=desc)
        
    plt.tight_layout()
    if DO_SAVE:
        plt.savefig('output/scaling_%s_%s.pdf'%(DIR, pCol), bbox_inches='tight')
